In [14]:
import numpy as np
import cv2
from imutils import paths
import imutils
import math
import collections as coll




def GetBlurredGrayScaleImage(imagePath):
    img = cv2.imread(imagePath)
    #img = ~img
    #img = cv2.imread('TriangleWithDot.jpg')
    img = cv2.resize(img, (800, 800))
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgGray = cv2.GaussianBlur(imgGray, (5, 5), 0)
    return imgGray




def DrawContours(contours, img):
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)

        # if too few curves skip this contour
        if len(approx) < 3:
            continue
        
        cv2.drawContours(img, [approx], 0, (0, 0, 255), 3)
        
def DrawContours(contours, img, color):
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)

        # if too few curves skip this contour
        if len(approx) < 3:
            continue
        
        cv2.drawContours(img, [approx], 0, color, 3)
        
def DrawContoursWithText(contours, img, text):
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)

        # if too few curves skip this contour
        if len(approx) < 3:
            continue
        
        cv2.drawContours(img, [approx], 0, (0, 0, 255), 3)
        x = approx.ravel()[0]
        y = approx.ravel()[1]
        cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255))

def DrawContoursWithTextInCenter(contourIDs, contours, img, text, color):
    for contourID in contourIDs:
        contour = contours[contourID]
        approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)

        # if too few curves skip this contour
        if len(approx) < 3:
            continue
        
        cv2.drawContours(img, [approx], 0, color, 3)
        
        # compute center of contour
        M = cv2.moments(contour)
        area = float(M["m00"])

        cX = 0
        cY = 0
        if area > 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
        cv2.putText(img, text, (cX, cY), cv2.FONT_HERSHEY_COMPLEX, 0.5, color)
        
def DrawContoursFromIDWithText(contourIDs, contours, img, text):
      
    for contourID in contourIDs:
        approx = cv2.approxPolyDP(contours[contourID], 0.01*cv2.arcLength(contours[contourID], True), True)

        # if too few curves skip this contour
        if len(approx) < 3:
            continue
        
        cv2.drawContours(img, [approx], 0, (0, 0, 255), 3)
        x = approx.ravel()[0]
        y = approx.ravel()[1]
        cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255))

        
def DrawContoursFromIDWithTextInCenter(contourIDs, contours, img, text):
    if(len(contourIDs) > 0):
        allX = 0
        allY = 0

        for contourID in contourIDs:
            approx = cv2.approxPolyDP(contours[contourID], 0.01*cv2.arcLength(contours[contourID], True), True)

            # if too few curves skip this contour
            if len(approx) < 3:
                continue

            cv2.drawContours(img, [approx], 0, (0, 0, 255), 3)
            x = approx.ravel()[0]
            y = approx.ravel()[1]
            allX = allX + x
            allY = allY + y

        x = int(allX / len(contourIDs))
        y = int(allY / len(contourIDs))
        cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255))
    
    

def DrawContoursFromID(contourIDs, contours, img, color):
    for contourID in contourIDs:
        approx = cv2.approxPolyDP(contours[contourID], 0.01*cv2.arcLength(contours[contourID], True), True)

        # if too few curves skip this contour
        if len(approx) < 3:
            continue
        
        cv2.drawContours(img, [approx], 0, color, 3)
        x = approx.ravel()[0]
        y = approx.ravel()[1]
        cv2.putText(img, str(contourID), (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, color)

        
def DrawGroupedContoursFromID(contourIDsList, contours, img):
    for group in contourIDsList:
        DrawContoursFromIDWithTextInCenter(group, contours, img, str(len(group)))


def DrawContoursWithArea(contours, hierarchy, img, minArea, maxArea):
    
    for component in zip(contours, hierarchy):
        contour = component[0]
        conHier = component[1]
        #approximate the number of polygonal curves
        approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)

        # if too few curves skip this contour
        if len(approx) < 3:
            continue

        # compute center of contour
        M = cv2.moments(contour)
        area = float(M["m00"])

        if area < minArea:
            continue
        if area > maxArea:
            continue
        else:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            
        
        # drawContours(imageToDrawOnto, contourOrApproximate, 
        #         indexOfContour (here always 0 since we are iterating over the contoursvia for), lineColor, lineThickness )
        cv2.drawContours(img, [approx], 0, (0, 0, 255), 3)
        x = approx.ravel()[0]
        y = approx.ravel()[1]

        # this gives inner most children since there are no child nodes
        if conHier[2] < 0:
            cv2.putText(img, str(len(conHier)), (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255))
            
        # this gives outer most parent since there are no child nodes
        if conHier[3] < 0:
            cv2.putText(img, str(0), (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255))
        
        #cv2.putText(img, conHier[2], (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255))
        # if we have three polygonal curves it is triangle
#         if len(approx) == 3:
#             cv2.putText(img, "Triangle", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255))
#         # Square
#         if len(approx) == 4:
#     #         # for checking if it square or rectangle
#     #         x, y, w, h = cv2.boundingRect(approx)
#     #         aspectRatio = float(w)/h
#     #         print(aspectRatio)
#     #         if aspectRatio >= 0.95 and aspectRatio <= 1.05:

#             cv2.putText(img, "Square", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255))
#         # Pentagon
#         if len(approx) == 5:
#             cv2.putText(img, "Pentagon", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255))

            

            
def getContoursWithArea(contours, hierarchy, minArea, maxArea):
    returnContours = []
    returnHierarchy = []
    
    for component in zip(contours, hierarchy):
        contour = component[0]
        conHier = component[1]
        
        #approximate the number of polygonal curves
        approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)

        # if too few curves skip this contour
        if len(approx) < 3:
            continue

        # compute center of contour
        M = cv2.moments(contour)
        area = float(M["m00"])

        if area == 0:
            continue
        elif area < minArea:
            continue
        elif area > maxArea:
            continue
        else:
            returnContours.append(contour)
            returnHierarchy.append(conHier)
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
    return returnContours, returnHierarchy

def getContourIDsWithArea(contours, hierarchy, minArea, maxArea):
    returnContours = []
    returnHierarchy = []
    
    i = -1
    for component in zip(contours, hierarchy):
        i += 1
        contour = component[0]
        conHier = component[1]
        
        #approximate the number of polygonal curves
        approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)

        # if too few curves skip this contour
        if len(approx) < 3:
            continue

        # compute center of contour
        M = cv2.moments(contour)
        area = float(M["m00"])

        if area == 0:
            continue
        elif area < minArea:
            continue
        elif area > maxArea:
            continue
        else:
            returnContours.append(i)
            returnHierarchy.append(conHier)
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
        
    return returnContours, returnHierarchy


def getDotsInOutline(outlineID, contours, hierarchy, minAreaDot, maxAreaDot):
    #print("Outline ID ", outlineID)
    
    # this returns a list of size contours where every contour id that is not a dot will be set to -1
    dotIDs = getDotIDsInOutline(outlineID, contours, hierarchy, minAreaDot, maxAreaDot)
    print("IDs: ", dotIDs )
    returnDotCon = []
    for i in range(0, len(dotIDs)):
        if dotIDs[i] > 0:
            returnDotCon.append(contours[dotIDs[i]])
    
    return returnDotCon



def getDotIDsInOutlineWithEccentricity(contourID, contours, hierarchy, minDotSize, maxDotSize, dotMinEcc, dotMaxEcc):
    
    returnContours = []
    skippedOutlineContourIDs = []
    
    if contourID > len(contours) or contourID > len(hierarchy):
        #print("ID is bigger than contours. -1 will be returned")
        #returnContours.append(-1)
        return returnContours, skippedOutlineContourIDs
    
    # get child contour of outline
    # return outline contour if there is no child contour -> make this recursive
    if hierarchy[contourID][2] > 0:
        
        skippedOutlineContourIDs.append(contourID)
        # get all children IDs
        childContourIDs = getAllContourIDsOfSameHierarchy(hierarchy[contourID][2], contours, hierarchy)
        #print("ChildContourIDS ", childContourIDs)
        
        # add children ids to return ids and return them
        for i in range(0, len(childContourIDs)):
            tmpRet, tmpOut = getDotIDsInOutlineWithEccentricity(childContourIDs[i], contours, hierarchy, minDotSize, maxDotSize, dotMinEcc, dotMaxEcc)
            returnContours.extend(tmpRet)
            skippedOutlineContourIDs.extend(tmpOut)
        return returnContours, skippedOutlineContourIDs
        
    else:
        # compute area of contour
        M = cv2.moments(contours[contourID])
        area = float(M["m00"])

        # return empty if area is too big or small for dot
        # return outline contour otherwise
        if area > maxDotSize:
            #print("Area is bigger")
            #returnContours.append(-1)
            return returnContours, skippedOutlineContourIDs
        elif area < minDotSize:
            #print("Area is smaller")
            #returnContours.append(-1)
            return returnContours, skippedOutlineContourIDs
        else:
            # need at least 5 points for ellipse
            if len(contours[contourID]) < 5:
                return returnContours, skippedOutlineContourIDs
            ellipse = cv2.fitEllipse(contours[contourID])
            width = ellipse[1][0]
            height = ellipse[1][1]
            ecc = 0
            if width < height:
                ecc = math.sqrt(1 - width**2 / height**2)
            else:
                ecc = math.sqrt(1 - height**2 / width**2)
            
            if ecc < dotMinEcc or ecc > dotMaxEcc:
                return returnContours, skippedOutlineContourIDs
            else:
                returnContours.append(contourID)
                return returnContours, skippedOutlineContourIDs



def getDotIDsInOutline(contourID, contours, hierarchy, minDotSize, maxDotSize):
    returnContours = []
    skippedOutlineContourIDs = []
    
    if contourID > len(contours) or contourID > len(hierarchy):
        #print("ID is bigger than contours. -1 will be returned")
        #returnContours.append(-1)
        return returnContours, skippedOutlineContourIDs
    
    # get child contour of outline
    # return outline contour if there is no child contour -> make this recursive
    if hierarchy[contourID][2] > 0:
        
        skippedOutlineContourIDs.append(contourID)
        # get all children IDs
        childContourIDs = getAllContourIDsOfSameHierarchy(hierarchy[contourID][2], contours, hierarchy)
        #print("ChildContourIDS ", childContourIDs)
        
        # add children ids to return ids and return them
        for i in range(0, len(childContourIDs)):
            tmpRet, tmpOut = getDotIDsInOutline(childContourIDs[i], contours, hierarchy, minDotSize, maxDotSize)
            returnContours.extend(tmpRet)
            skippedOutlineContourIDs.extend(tmpOut)
        return returnContours, skippedOutlineContourIDs
        
    else:
        # compute area of contour
        M = cv2.moments(contours[contourID])
        area = float(M["m00"])

        # return empty if area is too big or small for dot
        # return outline contour otherwise
        if area > maxDotSize:
            #print("Area is bigger")
            #returnContours.append(-1)
            return returnContours, skippedOutlineContourIDs
        elif area < minDotSize:
            #print("Area is smaller")
            #returnContours.append(-1)
            return returnContours, skippedOutlineContourIDs
        else:
            #print("Should return 0 contour")
            returnContours.append(contourID)
            return returnContours, skippedOutlineContourIDs



        
        
        
        
        
        
        
# this should return an array which contains subarrays which are the dots close to each other
# Example Form: [[0,1,2],[3,4],[5,8,9,10],[7]]
def getGroupedDotsIDs(contourIDs, contours, hierarchy, maxDist):
    # control that there are enough entries
    if len(contourIDs) < 2:
        return [contourIDs]
    
    # copy contourIDs into seperate list which will then be emptied over time
    IDs = contourIDs.copy()
    returnIDs = []
    
    
    # this is to keep track if all entires have been sorted into buckets
    addedIDs = []
    #while len(addedIDs) < len(contourIDs):
    while IDs:
        if len(IDs) < 1:
            break
        
        visited = []
        queue = []

        # get all contours whose center is within maxDIst
        queue = getIDsOfMaxDistance(IDs[0], IDs, contours, maxDist)
        
        # stop if there are no contours
        if len(queue) < 1:
            break

        
        # while there are still contours to check the distance
        while queue:
            # get first entry and add it to the visited list
            # then control for next one if it also has close nodes in list
            p = queue.pop(0)
            visited.append(p)
            addedIDs.append(p)
            
            # remove popped entry from IDs so that it wont be visited again
            if p in IDs:
                IDs.remove(p)
            
            # remove other closest entries from IDs since they are already in the queue
            for deleteID in queue:
                if deleteID in IDs:
                    IDs.remove(deleteID)
                
            if len(queue) > 0:
                queue.extend(getIDsOfMaxDistance(queue[0], IDs, contours, maxDist))
            
        # set(x) removes double entries 
        visited = list(set(visited))
        returnIDs.append(visited)
    
    
        
    return returnIDs
        
    
    # redo until list is empty

def getIDsOfMaxDistance(currentID, IDs, contours, maxDist):
    returnIDs = []
    
    currentCon = contours[currentID]
    
    # compute the center of the contour
    M = cv2.moments(currentCon)
    
    # float(M["m00"]) is area
    if float(M["m00"]) > 0:
        cX = float(M["m10"] / M["m00"])
        cY = float(M["m01"] / M["m00"])

        for ID in IDs:
            idContour = contours[ID]
            # compute the center of the contour
            idM = cv2.moments(idContour)
            if float(idM["m00"]) > 0:
                idCX = int(idM["m10"] / idM["m00"])
                idCY = int(idM["m01"] / idM["m00"])

                dist = getDistance(cX, cY, idCX, idCY)
                #print("Compared: ", str(currentCon), str(ID))

                if dist > maxDist:
                    continue
                else:
                    #print("Added: ", str(ID))
                    returnIDs.append(ID)
        
      
    return returnIDs

            
            
            
def getDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    #print("Dist ", str(dist))
    return dist

def getDistanceOfIDs(id1, id2, contours):
    con1 = contours[id1]
    con2 = contours[id2]
    
    # compute the center of the contour
    M1 = cv2.moments(con1)
    M2 = cv2.moments(con2)
    
    # float(M["m00"]) is area
    if float(M1["m00"]) > 0 and float(M2["m00"]) > 0:
        cX1 = float(M1["m10"] / M1["m00"])
        cY1 = float(M1["m01"] / M1["m00"])
        
        cX2 = float(M2["m10"] / M2["m00"])
        cY2 = float(M2["m01"] / M2["m00"])
        
        dist = getDistance(cX1, cY1, cX2, cY2)
        return dist
    
    return -1
    
def getAllContourIDsOfSameHierarchy(contourID, contours, hierarchies):
    returnContourIDs = []
    
    # append the given contour
    returnContourIDs.append(contourID)
    
    # get the next contour from the given
    nextContourID = contourID
    
    # while there is a next contour, get next and add the one to returnContours
    while hierarchies[nextContourID][0] > 0:
        nextContourID = hierarchies[nextContourID][0]
        
        returnContourIDs.append(nextContourID)

    return returnContourIDs  
    

def getAllContoursOfSameHierarchy(contourID, contours, hierarchies):
    returnContours = []
    returnHierarchies = []
    
    # append the given contour
    returnContours.append(contours[contourID])
    returnHierarchies.append(hierarchies[contourID])
    
    # get the next contour from the given
    nextContourID = contourID
    
    # while there is a next contour, get next and add the one to returnContours
    while hierarchies[nextContourID][0] > 0:
        nextContourID = hierarchies[nextContourID][0]
        
        returnContours.append(contours[nextContourID])
        returnHierarchies.append(hierarchies[nextContourID])

    return returnContours, returnHierarchies
        


# this returns only child Contours, so contours that have no other contour in them
def getChildContours(contours, hierarchy):
    returnContours = []
    returnHierarchy = []
    
    for component in zip(contours, hierarchy):
        contour = component[0]
        conHier = component[1]
        #approximate the number of polygonal curves
        approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)

        # if too few curves skip this contour
        if len(approx) < 3:
            continue

        # this gives inner most children since there are no child nodes
        if conHier[2] < 0:
            returnContours.append(contour)
            returnHierarchy.append(conHier)
    
    return returnContours, returnHierarchy
            

            
            
def trackChanged(x):
    pass
            
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result           
            

## this should return the ID and the image rotation from the array that contains the counts of dots
## 
def getTileIDAndShiftsFromGroupedArray(array, library):
    # this return -1 i the array is not found in the library
    ID = getArrayID(array, library)
    shifts = -1
    
    if ID >= 0:
        # library is in form: [[id0, [x,x,x,x,x,x,x,x]], [id1,[y,y,y,y,y,y,y,y]], ...]
        # the id hereby should be the index of the entry
        shifts = getShiftsOfArray(array, library[ID][1])
        # return degree from shift count and ID
#         if(shifts >= 0):
#             rotation = getRotationFromShifts(shifts)
            


    return ID, shifts
    
    
    # later: compare length of array against library to determine if something is occluded

    
def getSimpleRotationFromShifts(shifts):
    return shifts * 45    

def getZRotationFromShifts(shifts, centerArray, outlineID, contours):
    
    if shifts < 0:
        return -1
    if shifts > len(centerArray):
        return -1
    
    # get the centers coordinates
    outlineCenter = getCenterCoordinates([outlineID], contours)
    
    # for now without form
    # might be able without form since we will just compare the element that should be first
    zeroEntryCenter = centerArray[(len(centerArray) - shifts) % len(centerArray)]
    
    centerToZeroVec = [zeroEntryCenter[0] - outlineCenter[0], zeroEntryCenter[1] - outlineCenter[1]]
    print("zero angle ", getAngle(centerToZeroVec))
    return getAngle(centerToZeroVec)

def getXYRotationFromShiftsForSquare(shifts, centerArray):
    # Y rotation should be the same just for other corners comparison
    
    # get corners and compare front and back distance
#     5----6----7
#     |         |
#     4         0
#     |         |
#     3----2----1
    # compare diff(1,3) against diff(5, 7) for x rotation
    # compare diff(1,7) against diff(5, 3) for y rotation
    if len(centerArray) > 0:
        corner1 = centerArray[(len(centerArray) - shifts + 1) % len(centerArray)]
        corner3 = centerArray[(len(centerArray) - shifts + 3) % len(centerArray)]
        corner5 = centerArray[(len(centerArray) - shifts + 5) % len(centerArray)]
        corner7 = centerArray[(len(centerArray) - shifts + 7) % len(centerArray)]
        point0 = centerArray[(len(centerArray) - shifts + 0) % len(centerArray)]
        point2 = centerArray[(len(centerArray) - shifts + 2) % len(centerArray)]
        point4 = centerArray[(len(centerArray) - shifts + 4) % len(centerArray)]
        point6 = centerArray[(len(centerArray) - shifts + 6) % len(centerArray)]

        diff13 = getDiffOfPoints(corner1, corner3)
        diff57 = getDiffOfPoints(corner5, corner7)
        diff17 = getDiffOfPoints(corner1, corner7)
        diff35 = getDiffOfPoints(corner3, corner5)
        width = getDiffOfPoints(point0, point4)
        height = getDiffOfPoints(point2, point6)

        xRot = getRotationFromDifference(diff13, diff57)
        yRot = getRotationFromDifference(diff17, diff35)
        return xRot, yRot
    else:
        return 0, 0
    
def getRotationFromDifference(len1, len2):
    print("Need to test with rotated picture")
    if len1 > 0 and len2 > 0:
        ratio = len1 / len2
        print("ratio ", ratio)
        #return mapRange(ratio, 0, 2, -90, 90)
        if ratio > 1:
            # should be pos
            return mapRange(ratio, 1, 5, 0, 90)
        elif ratio < 1:
            # should be negative
            # will be between 0 and 1
            # needs to be between 0 and -90
            return mapRange(ratio, 0, 1, -90, 0)
        else:
            return 0
        #GK = len1 * 0.5 - len2 * 0.5
        #sin = GK / hypo
    # should get the angle depending on the difference of the two lines/lengths of the lines
    # need to keep track of positive and negative rotation
    # maybe start by dividing the two
    # if it is almost 1 -> rotation = 0
    # the more different -> rotation = XXX

    
def mapRange(x, valueMin, valueMax, targetMin, targetMax):
    if x > valueMax:
        x = valueMax
    if x < valueMin:
        x = valueMin
    
    x = x - valueMin
    x = x / (valueMax - valueMin)
    x = x * (targetMax - targetMin)
    x = x + targetMin
    return x
    
    
def getDiffOfPoints(point1, point2):
    diffVec = np.array([point1[0] - point2[0], point1[0] - point2[0]])
    return np.linalg.norm(diffVec)

def getArrayID(array, library):
    
    # library is in form: [[id0, [x,x,x,x,x,x,x,x]], [id1,[y,y,y,y,y,y,y,y]], ...]
    # the id hereby should be the index of the entry
    for entry in library:
        if checkIfArraysAreEqualWoOrder(array, entry[1]):
            return entry[0]
    
    print("No array found. -1 will be returned")
    return -1
            
    # go over entries in array and compare/check if they are in the entries (of entries) of the library
    # select the entry from the library that contains most entries from the array as best choice
    # return its ID
    

def getMostSimilarArrayIDsWoOrder(array, library, maxDiffVal):
    
    arrayIDs = []
    for i in range(0, len(library)):
        if getNumberOfDiffElementsWoOrder(array, library[i][1]) < maxDiffVal:
            arrayIDs.append(i)
    return arrayIDs
    
    
    return arrayID
    
def checkIfArraysAreEqualWoOrder(array1, array2):
    if len(array1) != len(array2):
        return False
    else:
        return coll.Counter(array1) == coll.Counter(array2)

    
def getSimilarityOfArrays(array1, array2):
    if len(array1) < len(array2):
        getNumberDiffElementsOfOrderedArrays(array1, array2)
    

def getNumberOfDiffElementsWoOrder(array1, array2):
    if len(array1) < len(array2):
        return list(coll.Counter(array2) - coll.Counter(array1))
    else:
        return list(coll.Counter(array1) - coll.Counter(array2))

        

    
def getNumberDiffElementsOfOrderedArrays(smallArray, largeArray):
    print("Not tested yet")
    diffElements = 0
    for i in range(0, len(smallArray)):
        if i + diffElements >= len(largeArray):
            break
            
        if smallArray[i] == largeArray[i + diffElements]:
            continue
        else:
            diffElements += 1
    
    return diffElements
    
    
    
def getShiftsOfArray(array, targetArray):
    shifts = 0
    
    # shift the array until it matches the targetArray
    for i in range(0, len(array)):
        if (np.array(array) == np.array(targetArray)).all():
            break
        #count the shifts
        
        shifts += 1
        array = np.roll(array, 1)
    
    #return shifts
    return shifts







def getSortedCountAndCenterArray(groupedArray, outlineID, contours):
    countArray = []
    angleArray = []
    centerArray = []
    
    if outlineID > len(contours):
        return []
    
    outlineX, outlineY = getCenterCoordinates([outlineID], contours)
    
    
    for group in groupedArray:
        dotsX, dotsY = getCenterCoordinates(group, contours)
        index, angleArray = getSortedAngleArrayAndInsertID(outlineX, outlineY, dotsX, dotsY, angleArray)
        countArray.insert(index, len(group))
        centerArray.insert(index, [dotsX, dotsY])
    print("angle array ", angleArray)
    print("sorted count array ", countArray)
    print("sorted center array ", centerArray)
    return countArray, centerArray

# this will return the id and the sorted array after cosine values
def getSortedAngleArrayAndInsertID(outlineX, outlineY, dotsX, dotsY, angleArray):
    
    vec = [dotsX - outlineX, dotsY - outlineY] #cv2.normalize(dotsC) - cv2.normalize(outlineC)
    
    angle = getAngle(vec)
    
    if len(angleArray) < 1:
        angleArray.append(angle)
        return 0, angleArray
    else:
        for i in range(0, len(angleArray)):
            if angle < angleArray[i]:
                angleArray.insert(i, angle)
                return i, angleArray
        angleArray.append(angle)
        return len(angleArray) - 1, angleArray


def getAngle(vec):
    #print("Vector before norm ", vec)
    #norm = np.linalg.norm(vec)
    #vec = vec / norm
    #print("Vector ", vec)
    arctan = np.arctan2(vec[1], vec[0])
    angle = np.degrees(arctan) % 360
    #print("Angle ", angle)
    return angle
        
def getCenterCoordinates(ids, contours):
    if len(ids) > 0:
        cX = 0
        cY = 0

        for ID in ids:
            M = cv2.moments(contours[ID])
            if float(M["m00"]) > 0:
                x = float(M["m10"] / M["m00"])
                y = float(M["m01"] / M["m00"])
                cX = cX + x
                cY = cY + y
            
        cX = int(cX / len(ids))
        cY = int(cY / len(ids))
        
        return cX, cY

    

    

def main():
    # creating trackbar to change contour size
    cv2.namedWindow('Contour Track Bar', cv2.WINDOW_NORMAL)
    minSizeDots = 'MinDot'
    maxSizeDots = 'MaxDot'
    minEcc = 'MinEcc'
    maxEcc = 'MaxEcc'
    minSizeOutline = 'MinOutline'
    maxSizeOutline = 'MaxOutline'
    maxDist = 'MaxDist'
    angleRot = 'Rot'
    
    bar = 'SizeBars'
    cv2.createTrackbar("MaxDot", "Contour Track Bar", 0, 1000, trackChanged)
    cv2.createTrackbar("MinDot", "Contour Track Bar", 0, 1000, trackChanged)
    cv2.createTrackbar("MinEcc", "Contour Track Bar", 0, 100, trackChanged)
    cv2.createTrackbar("MaxEcc", "Contour Track Bar", 0, 100, trackChanged)
    cv2.createTrackbar("MaxOutline", "Contour Track Bar", 0, 200000, trackChanged)
    cv2.createTrackbar("MinOutline", "Contour Track Bar", 0, 100000, trackChanged)
    cv2.createTrackbar("MaxDist", "Contour Track Bar", 0, 100, trackChanged)
    cv2.createTrackbar("Rot", "Contour Track Bar", 0, 359, trackChanged)
    
    
    # reading image
    img = cv2.imread('SquareRotated2.jpg')
    img = cv2.resize(img, (800, 800))
    imgGray = GetBlurredGrayScaleImage('SquareRotated2.jpg')
    
#     img = cv2.imread('SquareWithPointCode.jpg')
#     img = cv2.resize(img, (800, 800))
#     imgGray = GetBlurredGrayScaleImage('SquareWithPointCode.jpg')
    
    
    


    # absolute thresholding
    #_, thrash = cv2.threshold(imgGray, 45, 255, cv2.THRESH_BINARY_INV)

    # adaptive thresholding
    # mean thresholding
    # adaptiveThreshold(src, dst, 125, Imgproc.ADAPTIVE_THRESH_MEAN_C, Imgproc.THRESH_BINARY, number_of_surrounding_pixels(must be odd), 12)
    adapMeanThresh = cv2.adaptiveThreshold(imgGray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 55, 4)
    # gaussian thresholding
    adapGausThresh = cv2.adaptiveThreshold(imgGray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 55, 24)

    
    
    # library for TEST purpose
    # get later this library from external file and load it in once in the beginning
    idLibrary = [[0,[1, 4, 2, 1, 3, 2, 1, 3]], [1,[1,2,3,4,1,2,3,4]], [2,[1,1,1,1,1,1,1,1]]]

    while(True):
        # get track bar values
        minAreaDot = cv2.getTrackbarPos("MinDot", "Contour Track Bar")
        maxAreaDot = cv2.getTrackbarPos("MaxDot", "Contour Track Bar")
        minEcc = cv2.getTrackbarPos("MinEcc", "Contour Track Bar") * 0.01
        maxEcc = cv2.getTrackbarPos("MaxEcc", "Contour Track Bar") * 0.01
        minAreaOutline = cv2.getTrackbarPos("MinOutline", "Contour Track Bar")
        maxAreaOutline = cv2.getTrackbarPos("MaxOutline", "Contour Track Bar")
        maxDist = float(cv2.getTrackbarPos("MaxDist", "Contour Track Bar"))
        angle = float(cv2.getTrackbarPos("Rot", "Contour Track Bar"))
        
        # detect contour on threshold image
        detImg = adapGausThresh.copy()
        detImg = rotate_image(detImg, angle)
    
        # grab longest contour
        edged = cv2.Canny(detImg, 25, 325)
        contours, hierarchy = cv2.findContours(detImg, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        hierarchy = hierarchy[0] # get the actual inner list of hierarchy descriptions
        #print("Hierarchy", hierarchy)
        
        
        # this should later be changed for a list of outlines of the tiles
        #outlineID = cv2.getTrackbarPos("OutlineID", "Contour Track Bar")
        
        contourImage = img.copy()
        contourImage = rotate_image(contourImage, angle)
        
        
        # get contours of certain area
        
        # get dot contours
        dotCon, dotHier = getContoursWithArea(contours, hierarchy, minAreaDot, maxAreaDot)
        #get outline contours
        outlineConIDs, outlineHier = getContourIDsWithArea(contours, hierarchy, minAreaOutline, maxAreaOutline)
        alreadyVisited = []
        for outlineID in outlineConIDs:
            #if the outline was already visited from another outline skip it
            if outlineID in alreadyVisited:
                continue
            
            # get dots of only one outline
            #singleTileDotCon = getDotsInOutline(outlineID, contours, hierarchy, minAreaDot, maxAreaDot)
            #singleTileDotIDs, alreadyVisitedIDs = getDotIDsInOutline(outlineID, contours, hierarchy, minAreaDot, maxAreaDot)
            singleTileDotIDs, alreadyVisitedIDs = getDotIDsInOutlineWithEccentricity(outlineID, contours, hierarchy, minAreaDot, maxAreaDot,minEcc, maxEcc)
            
            alreadyVisited.extend(alreadyVisitedIDs)
            #print("Already visited ", alreadyVisited)
            #print("single dots ids ", singleTileDotIDs)
            
            # get only inner most nodes for dots
            #dotCon, dotHier = getChildContours(dotCon, dotHier)




            # draw outlines

            # draw dots
            #DrawContours(dotCon, contourImage)
            # draw outlines
            DrawContoursFromID([outlineID], contours, contourImage, (255, 0, 0))
            if len(singleTileDotIDs) > 0:
                print("")
                #DrawContoursWithText(singleTileDotCon, contourImage, str(outlineID))
                groupedDots = getGroupedDotsIDs(singleTileDotIDs, contours, hierarchy, maxDist)
                DrawGroupedContoursFromID(groupedDots, contours, contourImage)
                countArray, centerArray = getSortedCountAndCenterArray(groupedDots, outlineID, contours)
                tileID, shifts = getTileIDAndShiftsFromGroupedArray(countArray, idLibrary)
                exactRot = getZRotationFromShifts(shifts, centerArray, outlineID, contours)
                xRot, yRot = getXYRotationFromShiftsForSquare(shifts, centerArray)
    #             print("ID ", tileID)
    #             print("shifts ", shifts)
                print("Z rotation ", exactRot)
                print("XY rotation ", str(xRot), str(yRot))
                DrawContoursWithTextInCenter([outlineID], contours, contourImage, str(exactRot), (0, 255, 0))
                #print("Grouped Array", countArray)
                #DrawContoursFromID(singleTileDotIDs, contours, contourImage)
        
        
    
        cv2.imshow("DetectionImage", detImg)
        cv2.imshow("Contour Image", contourImage)
        cv2.imshow("edged", edged)
        if cv2.waitKey(1) == 27:
            break
        
    #cv2.waitKey(0)
    cv2.destroyAllWindows()

    
    
if __name__ == "__main__":
    main()
